In [ ]:
import os
import tarfile
import shutil
import dask
import requests
import time
import psutil

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from astrohack import holog
from astrohack.extract_holog import extract_holog
from astrohack.dio import _load_holog_file
from astrohack._utils._io import _make_ant_pnt_dict, _load_pnt_dict
%matplotlib widget
from dask.distributed import Client, LocalCluster
import matplotlib.pyplot as plt
from scipy import spatial

In [ ]:
# Only a single holography of VLA beam over scans 2,4,6,8,10,12,14,16,18,20,22 (ALMA can map an entire beam in a single scan).
holog_obs_description = {0 :{'scans':[2,4,6,8,10,12,14,16,18,20,22],'ant':{'EA25':['EA08', 'EA04', 'EA12', 'EA27', 'EA19', 'EA07', 'EA21', 'EA03', 'EA11', 'EA05', 'EA01', 'EA09', 'EA24', 'EA22', 'EA14', 'EA18', 'EA02']}}}
holog_obs_description['ddi'] = np.arange(16)

extract_holog(
    ms_name='ea25_cal_small.ms', 
    holog_name='ea25_cal_small', 
    holog_obs_dict=holog_obs_description,
    data_col='DATA',
    subscan_intent='MIXED',
    parallel=False,
    overwrite=True
)

In [ ]:
grid_size = None
cell_size = None
grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
chan_tolerance_factor = 0.005
reference_scaling_frequency = None #1.5*10**10 #None #In Hz
#spw_average if True users has to give reference_scaling_frequency
scan_average = True
holog(holog_file='ea25_cal_small.holog.zarr', padding_factor=50, 
      parallel=False,
      grid_size=grid_size, cell_size=cell_size, grid_interpolation_mode=grid_interpolation_mode,
      chan_average = chan_average, chan_tolerance_factor = chan_tolerance_factor,
      reference_scaling_frequency=reference_scaling_frequency,
      scan_average = scan_average,
     )

In [ ]:
%%time
from astrohack.panel import panel
image = 'ea25_cal_small'
#ddi = [1] 
ddi = None

panel(
    holog_image=image, 
    outfile='hack_panel',
    aipsdata=False, 
    unit='mm', 
    save_mask=True,
    save_phase=True,
    ddi=ddi
)